# AlexNet
---
## 构架概述
1. *卷积层（Convolutional Layer）：*

    - 卷积核（Filter）：卷积层的核心是卷积核（也称为滤波器），它是一个小的权重矩阵，通过在输入数据上滑动并执行卷积操作来提取特征。

    - 特征图（Feature Map）：卷积操作的结果称为特征图，它表示输入数据在特定卷积核下的响应。

    - 多通道：输入数据通常是多通道的（例如RGB图像有3个通道），卷积核也会相应地有多个通道，以处理不同通道的信息。

2. *激活函数（Activation Function）：*

    - 在卷积层之后通常会应用一个非线性激活函数（如ReLU），以引入非线性特性，增强模型的表达能力。

3. *池化层（Pooling Layer）：
*
    - 目的：池化层用于减少特征图的尺寸，从而减少计算量和参数数量，同时保留重要的特征。

    - 类型：常见的池化操作包括最大池化（Max Pooling）和平均池化（Average Pooling）。

4. *全连接层（Fully Connected Layer）：*

    - 在卷积和池化层之后，通常会连接一个或多个全连接层，用于将高维的特征图转换为一维的特征向量，并最终输出分类结果。

5. *归一化层（Normalization Layer）：*

    - 如局部响应归一化（Local Response Normalization, LRN），虽然现在较少使用，但在早期CNN中常见。

6. *Dropout：*

    - 在全连接层中使用Dropout技术，以减少过拟合。

## 典型结构

一个典型的CNN结构通常包括以下几个部分：

- *输入层：*接收原始图像数据。

- *卷积层：*提取图像的局部特征。

- *激活层：*引入非线性。

- *池化层：*减少特征图尺寸。

- *全连接层：*将特征图转换为分类结果。

- *输出层：*使用softmax等函数输出最终分类结果。



## 流程图 从LeNet（左）到AlexNet（右）

![流程图](https://zh-v2.d2l.ai/_images/alexnet.svg "LeNet to AlexNet")

AlexNet和LeNet的设计理念非常相似，但也存在显著差异。

1. AlexNet比相对较小的LeNet5要深得多。AlexNet由八层组成：五个卷积层、两个全连接隐藏层和一个全连接输出层。

2. AlexNet使用ReLU而不是sigmoid作为其激活函数

## 简易代码实现

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(
    # 这里使用一个11*11的更大窗口来捕捉对象。
    # 同时，步幅为4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
    nn.Linear(6400, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
    nn.Linear(4096, 10))

我们构造一个高度和宽度都为224的单通道数据，来观察每一层输出的形状。

In [2]:
X = torch.randn(1, 1, 224, 224)
for layer in net:
    X=layer(X)
    print(layer.__class__.__name__,'output shape:\t',X.shape)

Conv2d output shape:	 torch.Size([1, 96, 54, 54])
ReLU output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Conv2d output shape:	 torch.Size([1, 256, 26, 26])
ReLU output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 256, 12, 12])
ReLU output shape:	 torch.Size([1, 256, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 256, 5, 5])
Flatten output shape:	 torch.Size([1, 6400])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1,